In [ ]:
from pandas import DataFrame, read_csv
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from numpy import transpose
import numpy as np
from math import sqrt
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Input, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD,RMSprop,Adam
from tensorflow.keras.constraints import max_norm
from time import time
import seaborn as sns

In [ ]:
X_train_df = read_csv('X_train.csv', index_col=0)
X_val_df = read_csv('X_val.csv', index_col=0)
test_df = read_csv('X_test.csv', index_col=0)
y_train_df = read_csv('y_train.csv', index_col=0)
y_val_df = read_csv('y_val.csv', index_col=0)

In [ ]:
scaler_Y = StandardScaler()
scaler_Y.fit(y_train_df)

y_train = scaler_Y.transform(y_train_df)
y_val = scaler_Y.transform(y_val_df)

In [ ]:
scaler_Y = StandardScaler()
scaler_Y.fit(y_train_df)

y_train = scaler_Y.transform(y_train_df)
y_val = scaler_Y.transform(y_val_df)

In [ ]:
X_train = X_train_df.values.astype(float)
X_val = X_val_df.values.astype(float)
test = test_df.values.astype(float)

In [ ]:
nb_features = 50 # 24
X_train = X_train[:,:nb_features]
X_val = X_val[:,:nb_features]
test = test[:,:nb_features]

In [ ]:
def get_reg_model():
    return Sequential([
                    Input(shape=X_train.shape[1]),
                    Dense(1)
])

In [ ]:
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
from tensorflow.keras import initializers, regularizers, constraints

class Poly(Layer):
   
    def __init__(self, 
                 **kwargs):
        super(Poly, self).__init__(**kwargs)
        
          
    def build(self, input_shape):
        param_shape = list(input_shape[1:])
        self.alpha = self.add_weight(shape=param_shape,
                                     name='alpha',
                                     initializer='zeros',
#                                     regularizer=regularizers.l1_l2(0.001, 0.001),
                                     constraint=max_norm(3))

#         self.beta = self.add_weight(shape=param_shape,
#                                     name='beta',
#                                     initializer='zeros')
#         self.gamma = self.add_weight(shape=(1,),
#                                     name='gamma',
#                                     initializer='zeros')
        self.beta = 1
        
        #self.input_spec = InputSpec(ndim=len(input_shape))
        self.built = True

    def call(self, inputs, mask=None):
        return self.alpha * K.square(inputs) + self.beta * inputs
        gamma = tf.convert_to_tensor(mean_landmarks)
        # return K.sum(self.alpha * inputs**2 + self.beta * inputs, axis=1) + self.gamma

    def get_config(self):
        config = {
            'alpha_initializer': initializers.serialize(self.alpha_initializer),
            'alpha_regularizer': regularizers.serialize(self.alpha_regularizer),
            'alpha_constraint': constraints.serialize(self.alpha_constraint),
        }
        base_config = super(Poly, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def compute_output_shape(self, input_shape):
        return input_shape
        # return (1,)

In [ ]:
def get_poly_model():
    return Sequential([
                    Input(shape=X_train.shape[1]),
                    Poly(),
#                    Dense(32),
                    Dense(1, use_bias=True)
])

In [ ]:
model = get_poly_model()
#model = get_reg_model()
loss = 'mse'
LEARNING_RATE = 0.01

model.compile(loss=loss, optimizer=SGD(lr=LEARNING_RATE))

In [ ]:
BATCH_SIZE = X_train.shape[0] # computing the loss over the whole dataset
EPOCHS = 2000 # how many iterations over the whole dataset
t_0 = time()
#with tf.device('/device:GPU:0'):
history = model.fit(X_train, y_train,  validation_data=(X_val, y_val), epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=0)

print(time()-t_0, 's')

In [ ]:
model.summary()

In [ ]:
DataFrame(history.history).plot(figsize=(8, 5), logy=True)
plt.grid(True)
#plt.gca().set_ylim(0, 1)
plt.title('Model performance throughout training')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
y_p_scaled = model.predict(X_val)
y_p_scaled = y_p_scaled.reshape(y_p_scaled.shape[0])
y_p_val = scaler_Y.inverse_transform(y_p_scaled)

RMSLE = sqrt(mean_squared_error(y_val_df,y_p_val))
print('Validation RMSLE:', RMSLE)
y_p_scaled = model.predict(X_train)
y_p_scaled = y_p_scaled.reshape(y_p_scaled.shape[0])

y_p_train = scaler_Y.inverse_transform(y_p_scaled)

RMSLE = sqrt(mean_squared_error(y_train_df.SalePrice, y_p_train))
print('Train RMSLE:', RMSLE)

In [ ]:
model.get_weights()